### Checking the Dominance check for all the health variables in each of the cluster

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("KMeans_UMAP.csv")

df.head()

,General_hearing_condition,Had_high_blood_pressure,WBC,Haemoglobin,Platelete,Gender,Age,Race_Ethnicity,Country_of_Birth,Education_Level,...,Pulse,BODY_MEASURE_COMPOSITE,blood_macros,mean_steroid_ng_dl,balance_symptom_score,balance_impact_score,fall_risk_score,functional_difficulty_composite,Age_Group,umap_clusters
0,Good,Yes,6.5,14.0,241.0,Female,71.0,Non-Hispanic White,Born in 50 US states or Washington,High school graduate/GED or equivalent,...,69.333333,0.249125,-0.012353,-0.274090,0,0.0,0.0,Some difficulty,61-75,6
1,Good,No,6.8,13.9,264.5,Male,32.0,Mexican American,Unknown/NA,High school graduate/GED or equivalent,...,72.000000,0.318792,-0.049170,0.216401,0,0.0,0.0,No difficulty,31-45,0
2,Good,Yes,6.5,14.0,241.0,Female,65.0,Non-Hispanic Black,Born in 50 US states or Washington,High school graduate/GED or equivalent,...,69.333333,0.475114,-0.012353,-0.274090,0,0.0,0.0,Some difficulty,61-75,6
3,A little trouble,Yes,6.5,14.0,241.0,Male,74.0,Non-Hispanic White,Born in 50 US states or Washington,Some college or AA degree,...,69.333333,0.249125,-0.012353,-0.274090,0,0.0,0.0,Some difficulty,61-75,6
4,Good,Unknown/NA,9.3,12.9,439.0,Male,8.0,Other Race - Including Multi-Racial,Born in 50 US states or Washington,Unknown/NA,...,71.666667,-1.817027,-0.148207,-0.680624,0,0.0,0.0,Some difficulty,0-18,3


In [2]:
df.shape

(9442, 47)

In [3]:
df.columns

Index(['General_hearing_condition', 'Had_high_blood_pressure', 'WBC',
       'Haemoglobin', 'Platelete', 'Gender', 'Age', 'Race_Ethnicity',
       'Country_of_Birth', 'Education_Level', 'Marital_Status',
       'Household_Size', 'Income_to_Poverty_Ratio', 'Has_diabetes',
       'Takes_vitamin_supplements', 'Has_Disability', 'HDL_mg',
       'Has_Hepatitis', 'Covered_by_health_insurance', 'Tested_for_HIV_Virus',
       'General_health_condition', 'Received_Hepatitis_A_Vaccine',
       'Family_poverty_level_index', 'Has_Kidney_Failure', 'Had_Asthma',
       'Had_Arthritis', 'Had_heart_attack', 'Had_Thyroid',
       'Had_Liver_COndition', 'Had_Cancer', 'Teeth_and_gum_health',
       'Number_of_Moderate_Physical_activities_per_week',
       'Number_of_Vigorous_Physical_activities_per_week',
       'Number_of_hours_of_sleep', 'Cholestrol_level', 'SystolicBP',
       'DiastolicBP', 'Pulse', 'BODY_MEASURE_COMPOSITE', 'blood_macros',
       'mean_steroid_ng_dl', 'balance_symptom_score', 'balan

In [4]:
for col in df.columns:
    print(f"\n{col}")
    print(df[col].unique())


General_hearing_condition
['Good' 'A little trouble' 'Excellent' 'Moderate hearing trouble'
 'A lot of trouble' 'Deaf' "Don't know"]

Had_high_blood_pressure
['Yes' 'No' 'Unknown/NA']

WBC
[ 6.5   6.8   9.3   5.3   6.2   6.9   7.8   5.1   4.5   5.9   9.   10.8
  8.8   9.7   6.7   7.    5.5   7.3   5.4   5.7  15.6   7.6   8.3   4.1
  8.    4.9   6.3   9.2   7.5   7.9   6.4   7.2   8.5   8.1  10.3   4.7
  7.4  11.3   6.    6.45  9.1   5.    4.3   8.7  12.2   5.8   3.9   9.6
 10.5   6.6   5.6  10.   10.7   9.4   7.7   4.6   8.4   3.8   6.1  15.9
  4.4   4.2  15.1   7.1  15.3   4.8  10.2  12.   10.4   3.4   9.8   4.
  5.2   8.2  10.1  12.4  10.6   3.1  11.5  11.7  13.    9.9   9.5   8.9
  8.6   2.9  11.4  11.1   3.7   2.3  13.2  12.3  10.9  11.2  11.    3.2
  3.6   3.5   2.7  13.9   2.4   3.3  11.9  13.8  12.7  13.3  12.1  12.9
 14.5  13.4  13.5  18.   13.1  18.4  15.8  14.1   3.   11.8  17.4  12.8
 14.   12.6   2.5  11.6   2.6   2.8   2.2  17.5  13.6  14.2  12.5  15.4
 15.7  13.7  15.2  

In [14]:
def create_cluster_profile_dataframe(df):
    """
    Create a comprehensive cluster profiling dataframe
    Rows: Cluster labels
    Columns: All features with their dominant characteristics
    EXCLUDING socioeconomic and demographic features (including Age)
    """
    
    # Get all cluster labels
    clusters = sorted(df['umap_clusters'].unique())
    
    # Initialize results dictionary
    cluster_profiles = {}
    
    # Define categorical vs numerical columns (EXCLUDING socioeconomic/demographic)
    categorical_cols = [
        'General_hearing_condition', 'Had_high_blood_pressure',
        'Has_diabetes', 'Takes_vitamin_supplements', 'Has_Disability',
        'Has_Hepatitis', 'Tested_for_HIV_Virus',
        'General_health_condition', 'Received_Hepatitis_A_Vaccine',
        'Has_Kidney_Failure', 'Had_Asthma', 'Had_Arthritis', 'Had_heart_attack',
        'Had_Thyroid', 'Had_Liver_COndition', 'Had_Cancer', 'Teeth_and_gum_health',
        'functional_difficulty_composite'
    ]
    
    numerical_cols = [
        'WBC', 'Haemoglobin', 'Platelete',
        'HDL_mg', 'Cholestrol_level',
        'SystolicBP', 'DiastolicBP', 'Pulse', 'BODY_MEASURE_COMPOSITE',
        'blood_macros', 'mean_steroid_ng_dl', 'balance_symptom_score',
        'balance_impact_score', 'fall_risk_score', 'Number_of_hours_of_sleep',
        'Number_of_Moderate_Physical_activities_per_week',
        'Number_of_Vigorous_Physical_activities_per_week'
    ]
    
    # Define special columns that use Good/Bad/Worse instead of High/Medium/Low
    good_bad_worse_cols = ['balance_symptom_score', 'blood_macros', 'BODY_MEASURE_COMPOSITE']
    
    # Process each cluster
    for cluster in clusters:
        cluster_data = df[df['umap_clusters'] == cluster]
        cluster_profile = {}
        
        # Process CATEGORICAL columns
        for col in categorical_cols:
            if col in df.columns:
                # Get value counts and percentages
                value_counts = cluster_data[col].value_counts()
                total = len(cluster_data[col].dropna())
                
                if total > 0:
                    # Get top category and its percentage
                    top_category = value_counts.index[0]
                    top_percentage = (value_counts.iloc[0] / total) * 100
                    
                    # For binary-like columns (Yes/No), show Yes percentage
                    if col in ['Had_high_blood_pressure', 'Has_diabetes', 'Takes_vitamin_supplements',
                               'Has_Disability', 'Has_Hepatitis', 
                               'Tested_for_HIV_Virus', 'Has_Kidney_Failure', 'Had_Asthma',
                               'Had_Arthritis', 'Had_heart_attack', 'Had_Thyroid',
                               'Had_Liver_COndition', 'Had_Cancer']:
                        
                        yes_count = cluster_data[col].isin(['Yes']).sum()
                        yes_pct = (yes_count / total) * 100
                        cluster_profile[col] = f"{yes_pct:.1f}% Yes"
                    else:
                        # For other categorical, show dominant category
                        cluster_profile[col] = f"{top_category} ({top_percentage:.1f}%)"
                else:
                    cluster_profile[col] = "No data"
        
        # Process NUMERICAL columns
        for col in numerical_cols:
            if col in df.columns:
                # Remove missing values (including the weird outlier)
                clean_data = cluster_data[col].replace(5.39760535e-79, np.nan)
                clean_data = clean_data[clean_data != -1]  # Remove -1 missing indicators
                clean_data = clean_data.dropna()
                
                if len(clean_data) > 0:
                    # Calculate tertiles for the entire dataset (not just cluster)
                    full_data = df[col].replace(5.39760535e-79, np.nan)
                    full_data = full_data[full_data != -1].dropna()
                    
                    if len(full_data) > 0:
                        low_threshold = full_data.quantile(0.33)
                        high_threshold = full_data.quantile(0.67)
                        
                        # Get cluster mean
                        cluster_mean = clean_data.mean()
                        
                        # Categorize based on column type
                        if col in good_bad_worse_cols:
                            # Use Good/Bad/Worse for specific columns
                            if cluster_mean < low_threshold:
                                category = "Good"
                            elif cluster_mean > high_threshold:
                                category = "Worse"
                            else:
                                category = "Bad"
                            cluster_profile[col] = f"{category} ({cluster_mean:.2f})"
                        else:
                            # Use High/Medium/Low for other numerical columns
                            if cluster_mean < low_threshold:
                                category = "Low"
                            elif cluster_mean > high_threshold:
                                category = "High"
                            else:
                                category = "Medium"
                            cluster_profile[col] = f"{category} ({cluster_mean:.2f})"
                    else:
                        cluster_profile[col] = "No data"
                else:
                    cluster_profile[col] = "No data"
        
        cluster_profiles[f'Cluster_{cluster}'] = cluster_profile
    
    # Create DataFrame
    profile_df = pd.DataFrame(cluster_profiles).T
    
    return profile_df

# Usage
cluster_profiles_df = create_cluster_profile_dataframe(df)
cluster_profiles_df

,General_hearing_condition,Had_high_blood_pressure,Has_diabetes,Takes_vitamin_supplements,Has_Disability,Has_Hepatitis,Tested_for_HIV_Virus,General_health_condition,Received_Hepatitis_A_Vaccine,Has_Kidney_Failure,...,Pulse,BODY_MEASURE_COMPOSITE,blood_macros,mean_steroid_ng_dl,balance_symptom_score,balance_impact_score,fall_risk_score,Number_of_hours_of_sleep,Number_of_Moderate_Physical_activities_per_week,Number_of_Vigorous_Physical_activities_per_week
Cluster_0,Excellent (49.4%),8.2% Yes,1.7% Yes,1.5% Yes,11.5% Yes,0.2% Yes,20.2% Yes,Very good (37.8%),"Yes, at least 2 doses (35.3%)",0.8% Yes,...,Medium (72.68),Bad (0.23),Bad (-0.16),High (0.61),Worse (0.61),High (0.37),High (2.21),Medium (7.93),High (3.09),High (1.76)
Cluster_1,Good (54.8%),31.4% Yes,15.6% Yes,0.4% Yes,3.2% Yes,2.4% Yes,13.0% Yes,Good (43.9%),No doses (57.4%),1.1% Yes,...,Medium (71.30),Bad (0.16),Bad (-0.11),Medium (-0.05),Worse (0.82),High (0.31),High (0.82),Medium (7.55),Medium (2.01),Medium (0.64)
Cluster_2,Excellent (44.5%),24.3% Yes,6.3% Yes,1.5% Yes,4.6% Yes,0.9% Yes,21.2% Yes,Very good (41.8%),No doses (42.3%),1.4% Yes,...,Medium (72.04),Worse (0.39),Bad (-0.12),Medium (0.05),Worse (0.79),High (0.52),High (1.93),Low (7.46),High (3.15),High (2.09)
Cluster_3,Excellent (64.8%),0.0% Yes,0.3% Yes,0.8% Yes,0.4% Yes,0.3% Yes,0.2% Yes,Excellent (43.8%),"Yes, at least 2 doses (45.3%)",0.0% Yes,...,High (79.24),Good (-0.64),Good (-0.19),Medium (-0.13),Bad (0.00),Medium (0.00),High (0.01),High (8.05),High (5.29),Medium (0.43)
Cluster_4,Good (41.0%),38.9% Yes,12.1% Yes,3.5% Yes,17.9% Yes,1.9% Yes,29.3% Yes,Good (34.7%),No doses (52.4%),4.3% Yes,...,Medium (71.57),Bad (0.16),Worse (0.70),Medium (-0.03),Worse (0.82),High (0.70),High (3.28),Medium (7.63),Medium (2.61),High (1.10)
Cluster_6,Good (41.2%),52.2% Yes,19.4% Yes,3.0% Yes,3.5% Yes,1.3% Yes,14.1% Yes,Good (39.6%),No doses (71.9%),3.9% Yes,...,Medium (69.51),Bad (0.27),Bad (-0.14),Medium (-0.26),Worse (0.61),High (0.52),High (1.97),Medium (7.84),Medium (2.69),Medium (0.65)
Cluster_7,Good (29.2%),61.4% Yes,34.3% Yes,2.0% Yes,91.3% Yes,2.3% Yes,18.9% Yes,Fair (43.8%),No doses (70.1%),12.1% Yes,...,Medium (71.49),Worse (0.32),Bad (-0.14),Medium (-0.24),Worse (1.32),High (1.36),High (6.91),High (8.04),Medium (1.65),Medium (0.37)


In [15]:
cluster_profiles_df = create_cluster_profile_dataframe(df)
cluster_profiles_df

,General_hearing_condition,Had_high_blood_pressure,Has_diabetes,Takes_vitamin_supplements,Has_Disability,Has_Hepatitis,Tested_for_HIV_Virus,General_health_condition,Received_Hepatitis_A_Vaccine,Has_Kidney_Failure,...,Pulse,BODY_MEASURE_COMPOSITE,blood_macros,mean_steroid_ng_dl,balance_symptom_score,balance_impact_score,fall_risk_score,Number_of_hours_of_sleep,Number_of_Moderate_Physical_activities_per_week,Number_of_Vigorous_Physical_activities_per_week
Cluster_0,Excellent (49.4%),8.2% Yes,1.7% Yes,1.5% Yes,11.5% Yes,0.2% Yes,20.2% Yes,Very good (37.8%),"Yes, at least 2 doses (35.3%)",0.8% Yes,...,Medium (72.68),Bad (0.23),Bad (-0.16),High (0.61),Worse (0.61),High (0.37),High (2.21),Medium (7.93),High (3.09),High (1.76)
Cluster_1,Good (54.8%),31.4% Yes,15.6% Yes,0.4% Yes,3.2% Yes,2.4% Yes,13.0% Yes,Good (43.9%),No doses (57.4%),1.1% Yes,...,Medium (71.30),Bad (0.16),Bad (-0.11),Medium (-0.05),Worse (0.82),High (0.31),High (0.82),Medium (7.55),Medium (2.01),Medium (0.64)
Cluster_2,Excellent (44.5%),24.3% Yes,6.3% Yes,1.5% Yes,4.6% Yes,0.9% Yes,21.2% Yes,Very good (41.8%),No doses (42.3%),1.4% Yes,...,Medium (72.04),Worse (0.39),Bad (-0.12),Medium (0.05),Worse (0.79),High (0.52),High (1.93),Low (7.46),High (3.15),High (2.09)
Cluster_3,Excellent (64.8%),0.0% Yes,0.3% Yes,0.8% Yes,0.4% Yes,0.3% Yes,0.2% Yes,Excellent (43.8%),"Yes, at least 2 doses (45.3%)",0.0% Yes,...,High (79.24),Good (-0.64),Good (-0.19),Medium (-0.13),Bad (0.00),Medium (0.00),High (0.01),High (8.05),High (5.29),Medium (0.43)
Cluster_4,Good (41.0%),38.9% Yes,12.1% Yes,3.5% Yes,17.9% Yes,1.9% Yes,29.3% Yes,Good (34.7%),No doses (52.4%),4.3% Yes,...,Medium (71.57),Bad (0.16),Worse (0.70),Medium (-0.03),Worse (0.82),High (0.70),High (3.28),Medium (7.63),Medium (2.61),High (1.10)
Cluster_6,Good (41.2%),52.2% Yes,19.4% Yes,3.0% Yes,3.5% Yes,1.3% Yes,14.1% Yes,Good (39.6%),No doses (71.9%),3.9% Yes,...,Medium (69.51),Bad (0.27),Bad (-0.14),Medium (-0.26),Worse (0.61),High (0.52),High (1.97),Medium (7.84),Medium (2.69),Medium (0.65)
Cluster_7,Good (29.2%),61.4% Yes,34.3% Yes,2.0% Yes,91.3% Yes,2.3% Yes,18.9% Yes,Fair (43.8%),No doses (70.1%),12.1% Yes,...,Medium (71.49),Worse (0.32),Bad (-0.14),Medium (-0.24),Worse (1.32),High (1.36),High (6.91),High (8.04),Medium (1.65),Medium (0.37)


In [16]:
# Or to see all unique values across all columns
for col in cluster_profiles_df.columns:
    print(f"\n{col}:")
    print(cluster_profiles_df[col].unique())


General_hearing_condition:
['Excellent (49.4%)' 'Good (54.8%)' 'Excellent (44.5%)'
 'Excellent (64.8%)' 'Good (41.0%)' 'Good (41.2%)' 'Good (29.2%)']

Had_high_blood_pressure:
['8.2% Yes' '31.4% Yes' '24.3% Yes' '0.0% Yes' '38.9% Yes' '52.2% Yes'
 '61.4% Yes']

Has_diabetes:
['1.7% Yes' '15.6% Yes' '6.3% Yes' '0.3% Yes' '12.1% Yes' '19.4% Yes'
 '34.3% Yes']

Takes_vitamin_supplements:
['1.5% Yes' '0.4% Yes' '0.8% Yes' '3.5% Yes' '3.0% Yes' '2.0% Yes']

Has_Disability:
['11.5% Yes' '3.2% Yes' '4.6% Yes' '0.4% Yes' '17.9% Yes' '3.5% Yes'
 '91.3% Yes']

Has_Hepatitis:
['0.2% Yes' '2.4% Yes' '0.9% Yes' '0.3% Yes' '1.9% Yes' '1.3% Yes'
 '2.3% Yes']

Tested_for_HIV_Virus:
['20.2% Yes' '13.0% Yes' '21.2% Yes' '0.2% Yes' '29.3% Yes' '14.1% Yes'
 '18.9% Yes']

General_health_condition:
['Very good (37.8%)' 'Good (43.9%)' 'Very good (41.8%)'
 'Excellent (43.8%)' 'Good (34.7%)' 'Good (39.6%)' 'Fair (43.8%)']

Received_Hepatitis_A_Vaccine:
['Yes, at least 2 doses (35.3%)' 'No doses (57.4%)' 'No d

# Cluster Profiling Methodology

## Overview
We created a comprehensive cluster profiling dataframe where each row represents a patient archetype (cluster) and each column represents a health feature. This allows us to understand the dominant characteristics of each patient group.

## Feature Categorization Approach

### 1. Categorical/Binary Health Conditions

**Method**: Percentage of "Yes" responses

For binary health conditions (e.g., Had_high_blood_pressure, Has_diabetes, Has_Asthma), we calculate:

% Yes = (Number of "Yes" responses in cluster / Total non-missing responses in cluster) × 100

**Rationale**: 
- This gives us the prevalence rate of each condition within each cluster
- Allows easy comparison across clusters (e.g., "Cluster 2 has 65% diabetes prevalence vs 12% in Cluster 5")
- Clinically meaningful - directly represents disease burden in each patient archetype

**Example Output**: "65.3% Yes" means 65.3% of patients in that cluster have the condition

### 2. Multi-Category Features

**Method**: Dominant category with percentage

For ordinal/multi-category features (e.g., General_health_condition, Teeth_and_gum_health), we report:

Most frequent category (% of responses)

**Rationale**:
- Captures the most representative state for that cluster
- Shows both the category and how dominant it is
- Helps identify if a cluster is homogeneous (high %) or heterogeneous (lower %)

**Example Output**: "Good (72.4%)" means 72.4% of patients in that cluster reported "Good" health

### 3. Numerical Features: Clinical Biomarkers

#### Standard Approach (High/Medium/Low)

For most clinical measurements (WBC, Haemoglobin, Cholesterol, BP, etc.):

**Method**: Tertile-based categorization using population-level thresholds

- Low: Mean < 33rd percentile of overall population
- Medium: Mean between 33rd and 67th percentile
- High: Mean > 67th percentile

**Why High is Preferred for Some Features**:

| Feature | High is Better Because... |
|---------|---------------------------|
| Haemoglobin | Higher values indicate better oxygen-carrying capacity; low = anemia |
| HDL_mg | "Good cholesterol" - higher protects against heart disease |
| Platelete | Higher (within normal range) indicates better clotting function |

**Why Low is Preferred for Other Features**:

| Feature | Low is Better Because... |
|---------|--------------------------|
| SystolicBP/DiastolicBP | Lower BP reduces cardiovascular risk; high BP = hypertension |
| Cholestrol_level | Lower total cholesterol reduces atherosclerosis risk |
| WBC | Extremely high WBC can indicate infection or blood disorders |
| Pulse | Lower resting heart rate often indicates better cardiovascular fitness |

**Example Output**: "High (156.2)" for Cholesterol means the cluster has elevated cholesterol (mean = 156.2 mg/dL)

#### Special Approach: Good/Bad/Worse

For composite health scores and functional assessments, we use Good/Bad/Worse instead:

**Applied to**:
- balance_symptom_score
- blood_macros (composite of blood test results)
- BODY_MEASURE_COMPOSITE (composite of BMI, waist, weight)

**Method**: 

- Good: Mean < 33rd percentile (lower scores = better health)
- Bad: Mean between 33rd and 67th percentile
- Worse: Mean > 67th percentile (higher scores = worse health)

**Rationale**:
- These are symptom/problem scores where higher values indicate worse health outcomes
- "Good/Bad/Worse" terminology is more clinically intuitive than "Low/Medium/High" for health status
- balance_symptom_score: Higher score = more balance problems → Worse
- BODY_MEASURE_COMPOSITE: Higher values often indicate obesity/metabolic issues → Worse
- blood_macros: Composite standardized score where extremes indicate abnormalities → Worse

**Example Output**: "Worse (2.45)" for balance_symptom_score means this cluster has more severe balance impairments

## Why Population-Level Thresholds?

We use tertiles from the entire dataset rather than cluster-specific ranges because:

1. **Enables Cross-Cluster Comparison**: All clusters evaluated on the same scale
2. **Clinical Context**: A cluster's "High" cholesterol is genuinely high relative to the full patient population
3. **Avoids Relativism**: Prevents a cluster with uniformly poor health from appearing "Medium" when they're actually all at high risk

## Exclusions: Socioeconomic & Demographic Features

We excluded the following to focus purely on clinical/health characteristics:
- Gender, Race_Ethnicity, Country_of_Birth
- Education_Level, Marital_Status, Age, Age_Group
- Household_Size, Income_to_Poverty_Ratio, Family_poverty_level_index
- Covered_by_health_insurance

**Rationale**: 
- Focus on medical/health-based patient archetypes rather than demographic segmentation
- Reduces bias in cluster interpretation by centering on modifiable health factors
- Allows subsequent bias detection analysis to examine how clusters distribute across protected demographics

## Summary

This methodology provides:
- **Interpretable cluster profiles** using clinically meaningful categories
- **Consistent evaluation** across all patient archetypes using population baselines
- **Direction-aware categorization** that reflects whether high/low values are desirable
- **Focus on health outcomes** rather than demographic characteristics

Each cluster profile can now be read as: "This patient archetype has X% diabetes prevalence, High cholesterol, Good balance scores, and Medium blood pressure" - creating actionable clinical insights for personalized care strategies.